# Explore CMIP6 AMOC data

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import xarray as xr
from data_loading_functions import *
import matplotlib.gridspec as gridspec

datadir = '../../Other_data/AMOC_CMIP6/'

In [ ]:
h2o_rho = 1035 # kg/m**3

file_list = [ f.name for f in os.scandir(datadir) if f.is_file()]
models = {}
col_names = ["model", "member", "AMOC year 1 (Sv)", "AMOC year 150 (Sv)", "perc. change in year 150"]

df = pd.DataFrame([], columns = col_names)

for file in file_list:
    model_name = file.rsplit('_')[1];
    member = file.rsplit('_')[-1][:-3]

    model_ds = xr.load_dataset(datadir + file)
    amoc_strength = model_ds.amoc_a4x/h2o_rho/10**6 # in Sv
    amoc_year1 = amoc_strength[0].values; amoc_year150 = amoc_strength[150-1].values
    amoc_change = np.round((1 - amoc_year150/amoc_year1)*100,2) # % decrease from year 1 to year 150
    
    if model_name not in models:
        models[model_name] = []
    
    models[model_name].append(member)
    df_row = pd.DataFrame([[model_name, member, np.round(amoc_year1,2), np.round(amoc_year150,2), amoc_change]], columns=col_names)
    # Append the new row
    df = pd.concat([df, df_row], ignore_index=True)
    
#df.to_csv('../cmip6_abrupt-4xCO2_amoc_table.csv', index=False)
df

In [3]:
rmse_df = pd.read_table('../cmip6_abrupt-4xCO2_rmse_table.csv', sep=',')
all_joined = pd.merge(df, rmse_df, on=['model', 'member'], how='outer')
all_sorted = all_joined.sort_values(by = '% change2')
matching_rows = pd.merge(df, rmse_df, on=['model', 'member'], how='inner')

all_above3perc = all_joined[all_joined['% change2'] < -3]
matching_above3perc = matching_rows[matching_rows['% change2'] < -3]

all_neg_rmse = all_joined[all_joined['% change2'] < 0]
matching_neg_rmse = matching_rows[matching_rows['% change2'] < 0]

all_below3perc = all_joined[all_joined['% change2'] > -3]
matching_below3perc = matching_rows[matching_rows['% change2'] > -3]

#sorted_models = only_neg_rmse.sort_values(by = '% change2')

In [ ]:

matching_above3perc.sort_values(by = '% change2')

In [ ]:
matching_above3perc

In [ ]:
pd.set_option('display.max_rows', None)
#all_sorted[:26]

In [ ]:
fig = plt.figure(figsize=(10, 10))
gs = gridspec.GridSpec(3, 2, height_ratios=[1, 1, 1], width_ratios=[1, 1])

# Top plot (spans both columns)
ax1 = plt.subplot(gs[0, :])
ax = ax1;
ax.scatter(matching_rows["% change2"], matching_rows["AMOC year 1 (Sv)"]-matching_rows["AMOC year 150 (Sv)"], color = 'gray')
ax.scatter(matching_above3perc["% change2"], matching_above3perc["AMOC year 1 (Sv)"]-matching_above3perc["AMOC year 150 (Sv)"], color = 'black')
ax.set_ylabel('AMOC year 1 - AMOC year 150 (Sv)')
ax.set_xlabel('% change in RMSE')
ax.grid()
#ax.axvline(0, color = 'black')
ax.axvline(-3.0, color = 'black')
ax.set_title('Abrupt-4xCO2 AMOC change vs improvement with oscillations')
ax.text(0.01, 1.02, 'a)', transform=ax.transAxes, fontsize = 12, fontweight = 'bold');


# middle left plot
ax2 = plt.subplot(gs[1, 0])
ax = ax2;
# set water density
h2o_rho = 1035 # kg/m**3
for (i,model) in enumerate(matching_above3perc['model']):
    member = matching_above3perc['member'].iloc[i]
    model_files = [file for file in file_list if model in file]
    member_file = [file for file in model_files if member in file]
    model_ds = xr.load_dataset(datadir + member_file[0])
    (model_ds.amoc_a4x/h2o_rho/10**6).plot()
ax.set_xlim(0,150)
ax.set_ylim(0,35)
ax.grid()
ax.set_ylabel('Abrupt-4xCO2 AMOC (Sv)')
ax.set_xlabel('Year')
ax.set_title('Models with RMSE change < -3%')
ax.text(0.01, 1.02, 'b)', transform=ax.transAxes, fontsize = 12, fontweight = 'bold');


# middle right plot
ax3 = plt.subplot(gs[1, 1])
ax = ax3;
# set water density
h2o_rho = 1035 # kg/m**3
for (i,model) in enumerate(matching_below3perc['model']):
    member = matching_below3perc['member'].iloc[i]
    model_files = [file for file in file_list if model in file]
    member_file = [file for file in model_files if member in file]
    model_ds = xr.load_dataset(datadir + member_file[0])
    (model_ds.amoc_a4x/h2o_rho/10**6).plot()
ax.set_xlim(0,150)
ax.set_ylim(0,35)
ax.grid()
ax.set_ylabel('Abrupt-4xCO2 AMOC (Sv)')
ax.set_xlabel('Year')
ax.set_title('Models with RMSE change > -3%')
ax.text(0.01, 1.02, 'c)', transform=ax.transAxes, fontsize = 12, fontweight = 'bold');


# Bottom left plot
ax4 = plt.subplot(gs[2, 0])
ax = ax4;

for (i,model) in enumerate(all_above3perc['model']):
    member = all_above3perc['member'].iloc[i]
    data = load_anom(model, 'abrupt-4xco2', member, length_restriction = 150)
    ax.plot(data['tas'].values, color = 'lightgray')
    # plots all models in gray

# then models with amoc data in color:
for (i,model) in enumerate(matching_above3perc['model']):
    member = matching_above3perc['member'].iloc[i]
    data = load_anom(model, 'abrupt-4xco2', member, length_restriction = 150)
    ax.plot(data['tas'].values)

ax.set_xlim(0,150)
#ax.set_ylim(0,9)
ax.grid()
ax.set_ylabel('Temperature response (°C)')
ax.set_xlabel('Year')
ax.text(0.01, 1.02, 'd)', transform=ax.transAxes, fontsize = 12, fontweight = 'bold');



# Bottom right plot
ax5 = plt.subplot(gs[2, 1])
ax = ax5;
for (i,model) in enumerate(all_below3perc['model']):
    member = all_below3perc['member'].iloc[i]
    data = load_anom(model, 'abrupt-4xco2', member, length_restriction = 150)
    ax.plot(data['tas'].values, color = 'lightgray')
    # plots all models in gray

# then models with amoc data in color:
for (i,model) in enumerate(matching_below3perc['model']):
    member = matching_below3perc['member'].iloc[i]
    data = load_anom(model, 'abrupt-4xco2', member, length_restriction = 150)
    ax.plot(data['tas'].values)

ax.set_xlim(0,150)
#ax.set_ylim(0,9)
ax.grid()
ax.set_ylabel('Temperature response (°C)')
ax.set_xlabel('Year')
ax.text(0.01, 1.02, 'e)', transform=ax.transAxes, fontsize = 12, fontweight = 'bold');



# Adjust spacing between subplots
plt.subplots_adjust(hspace=0.3, wspace=0.3)

#plt.savefig('../amoc_vs_temp_responses.pdf', format='pdf', dpi=600, bbox_inches="tight") 

